In [5]:
import joblib

# Load the trained model
model = joblib.load("speech_non_speech_rf.joblib")


c:\Users\Hp\Desktop\college project\SpeachNonSpeachDeticition\myenv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Hp\Desktop\college project\SpeachNonSpeachDeticition\myenv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [19]:
import librosa

# Load an audio file (mono, 16kHz recommended)
audio_path = "voice.wav"  # <-- change this to your file
y, sr = librosa.load(audio_path, sr=16000)


C:\Users\Hp\AppData\Local\Temp\ipykernel_20720\4294088006.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000)
c:\Users\Hp\Desktop\college project\SpeachNonSpeachDeticition\myenv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [20]:
import numpy as np

def extract_features(y, sr):
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    zcr = librosa.feature.zero_crossing_rate(y)
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    
    # Take mean of each feature
    features = np.hstack([
        np.mean(mfccs, axis=1),
        np.mean(zcr),
        np.mean(centroid),
        np.mean(rolloff)
    ])
    return features


In [22]:
print("Expected features:", model.n_features_in_)


Expected features: 78


In [23]:
X_test = extract_features(y, sr)
print("Extracted features shape:", X_test.shape)


Extracted features shape: (16,)


In [24]:
import numpy as np
import librosa

def extract_features(y, sr):
    # --- Core spectral features ---
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=26).T, axis=0)           # 26
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)              # 12
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr).T, axis=0)[:26]         # 26
    contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr).T, axis=0)      # 7
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr).T, axis=0) # 6
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))                             # 1

    # --- Combine ---
    features = np.hstack([mfccs, chroma, mel, contrast, tonnetz, zcr])
    return features


In [25]:
X_test = extract_features(y, sr).reshape(1, -1)
print("Extracted features shape:", X_test.shape)

prediction = model.predict(X_test)
print("🎤 Speech detected" if prediction[0] == 1 else "🔇 Non-speech detected")


Extracted features shape: (1, 78)
🔇 Non-speech detected
